In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('friends').getOrCreate()

lines = spark.sparkContext.textFile('fakefriends.csv')

In [4]:
lines.take(5)

['0,Will,33,385',
 '1,Jean-Luc,26,2',
 '2,Hugh,55,221',
 '3,Deanna,40,465',
 '4,Quark,68,21']

In [5]:
from pyspark.sql import DataFrame, Row
def process_line(line):
    
    line = line.split(',')
    
    line = Row(ID = int(line[0]), name = line[1],
               age = int(line[2]), n_friends = int(line[3]))
    
    return line

rdd = lines.map(process_line)
schemaPeople = spark.createDataFrame(rdd).cache()

In [9]:
schemaPeople.groupBy('name').count().limit(10).show()

+--------+-----+
|    name|count|
+--------+-----+
|    Data|   25|
| Beverly|   20|
|    Hugh|   20|
|   Dukat|   21|
|Jean-Luc|   16|
|     Nog|   22|
|     Odo|   15|
|  Kasidy|   19|
|  Guinan|   12|
|   Leeta|   15|
+--------+-----+



In [17]:
schemaPeople.createOrReplaceTempView('people')
spark.sql('SELECT name, COUNT(*) FROM people GROUP BY name LIMIT 10').show()

+--------+--------+
|    name|count(1)|
+--------+--------+
|    Data|      25|
| Beverly|      20|
|    Hugh|      20|
|   Dukat|      21|
|Jean-Luc|      16|
|     Nog|      22|
|     Odo|      15|
|  Kasidy|      19|
|  Guinan|      12|
|   Leeta|      15|
+--------+--------+



In [27]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('friends').getOrCreate()

lines = spark.sparkContext.textFile('fakefriends.csv')
rdd = (
    lines
    .map(lambda x: x.split(',')) \
    .map(lambda x: Row(ID = int(x[0]), name = x[1], age = int(x[2]), n_friends = int(x[3])))
)
# [Row(ID=0, name='Will', age=33, n_friends=385),
#  Row(ID=1, name='Jean-Luc', age=26, n_friends=2),
#  Row(ID=2, name='Hugh', age=55, n_friends=221)]

schemaPeople = spark.createDataFrame(rdd).cache()
# schemaPeople.limit(5).show()
# +---+--------+---+---------+
# | ID|    name|age|n_friends|
# +---+--------+---+---------+
# |  0|    Will| 33|      385|
# |  1|Jean-Luc| 26|        2|
# |  2|    Hugh| 55|      221|
# |  3|  Deanna| 40|      465|
# |  4|   Quark| 68|       21|

schemaPeople.limit(5).show()

+---+--------+---+---------+
| ID|    name|age|n_friends|
+---+--------+---+---------+
|  0|    Will| 33|      385|
|  1|Jean-Luc| 26|        2|
|  2|    Hugh| 55|      221|
|  3|  Deanna| 40|      465|
|  4|   Quark| 68|       21|
+---+--------+---+---------+



In [40]:
schemaPeople.createOrReplaceTempView('people')
spark.sql('SELECT age, CAST(ROUND(AVG(n_friends)) AS INT) AS total_friends FROM people GROUP BY age ORDER BY age LIMIT 10').show()

+---+-------------+
|age|total_friends|
+---+-------------+
| 18|          343|
| 19|          213|
| 20|          165|
| 21|          351|
| 22|          206|
| 23|          246|
| 24|          234|
| 25|          197|
| 26|          242|
| 27|          228|
+---+-------------+



In [48]:
schemaPeople.groupby('age').sum('n_friends').show()

+---+--------------+
|age|sum(n_friends)|
+---+--------------+
| 26|          4115|
| 29|          2591|
| 65|          1491|
| 54|          3615|
| 19|          2346|
| 22|          1445|
| 34|          1473|
| 50|          1273|
| 57|          3106|
| 43|          1614|
| 32|          2287|
| 31|          2138|
| 39|          1185|
| 25|          2172|
| 68|          2696|
| 58|          1282|
| 27|          1825|
| 63|          1536|
| 56|          1840|
| 51|          2115|
+---+--------------+
only showing top 20 rows

